# Predicción sobre lealtad de clientes de BetaBank

# Obetivo: Crear un modelo confiable que prediga si un cliente dejará al Banco

## Preparación de los datos

### Carga de datos y librerias

In [1]:
# Vamos a cargar los datos a utilizar

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

data = pd.read_csv('Churn.csv')

print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1   

In [2]:
# Vamos a trabajar con la columna 'Tenure' que tiene valores ausentes; debemos buscar que hacer con esos valores ausentes

data_null = data[data['Tenure'].isna()]
print(data_null.head())
print(data_null['IsActiveMember'].value_counts(dropna=False))
print(data_null['Exited'].value_counts(dropna=False))
print(data_null['Geography'].value_counts(dropna=False))

    RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
30         31    15589475    Azikiwe          591     Spain  Female   39   
48         49    15766205        Yin          550   Germany    Male   38   
51         52    15768193  Trevisani          585   Germany    Male   36   
53         54    15702298   Parkhill          655   Germany    Male   41   
60         61    15651280     Hunter          742   Germany    Male   35   

    Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
30     NaN       0.00              3          1               0   
48     NaN  103391.38              1          0               1   
51     NaN  146050.97              2          0               0   
53     NaN  125561.97              1          0               0   
60     NaN  136857.00              1          0               0   

    EstimatedSalary  Exited  
30        140469.38       1  
48         90878.13       0  
51         86424.57       0  
53        164040.94 

### Procesamiento de valores nulos

In [3]:
# Vamos a calcular la media de Tenure para llenar aquellas celdas con valores ausentes

ternure_mean_1 = (data[(~data['Tenure'].isna())&(data['Exited']==1)]['Tenure']).mean()
ternure_mean_0 = (data[(~data['Tenure'].isna())&(data['Exited']==0)]['Tenure']).mean()
print(ternure_mean_1)
print(ternure_mean_0)

data['Tenure'] = data['Tenure'].fillna(ternure_mean_0)
print(data.info())

4.901833872707659
5.022246787342822
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


### Evaluación del contenido

In [4]:
# Vamos a ver la información de los usuarios donde la columna "Balance" es 0
print((data[data['Balance']==0])['Exited'].value_counts())
print((data[data['Balance']==0])['IsActiveMember'].value_counts())

Exited
0    3117
1     500
Name: count, dtype: int64
IsActiveMember
1    1873
0    1744
Name: count, dtype: int64


In [5]:
# Vamos a transformar nuestras caracteristicas categóricas en características numéricas para poder entrenar nuestro modelo
data_ohe = data.drop('Surname', axis=1)
#categories = ['Geography','Gender']
data_ohe = pd.get_dummies(data_ohe, drop_first=True)
print(data_ohe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   RowNumber          10000 non-null  int64  
 1   CustomerId         10000 non-null  int64  
 2   CreditScore        10000 non-null  int64  
 3   Age                10000 non-null  int64  
 4   Tenure             10000 non-null  float64
 5   Balance            10000 non-null  float64
 6   NumOfProducts      10000 non-null  int64  
 7   HasCrCard          10000 non-null  int64  
 8   IsActiveMember     10000 non-null  int64  
 9   EstimatedSalary    10000 non-null  float64
 10  Exited             10000 non-null  int64  
 11  Geography_Germany  10000 non-null  bool   
 12  Geography_Spain    10000 non-null  bool   
 13  Gender_Male        10000 non-null  bool   
dtypes: bool(3), float64(3), int64(8)
memory usage: 888.8 KB
None


### Primera conclusión 

Antes de comenzar cualquier análisis, entrenamiento o predicción, bemeos de asegurar que nuestros datos son adecuados para comenzar con estas actividades y que entendemos el contenido y estructura de nuestra fuente datos y la información que nos da.

Por eso, en esta primera parte vimos que la columna "Tenure" presentaba poco menos de 1,000 celdas con valores nulos. Lo más sencillo era eliminar aquellas filas, sin embargo nos iba a hacer perder casi un 10% de la información de nuestro data set fuente. Por eso fue que opte por calcular la media de los valores de la columna 'Tenure' y rellenar esas celdas con valores nulos por la media de las demás obersvaciones. También se validó que, tomando en cuenta el contexto del caso, me parecio un poco extraño que tuvieramos Usuarios con Balance 0.0, por lo que quise encontrar alguna coincidencia con otras columnas, por ejemplo que auqellos usuarios con Balnce 0 fuese Usuarios que ya habiand ejado al banco o que no estuvieran activos. Sin embargo, no encontre alguna relación entre estas columnas y no realicé ningun otro procesamiento con los datos.

Por último, antes de seguir, implemente la codificación OHE para las columnas de variables categóricas como "Geography" y "Gender" or lo que se crearon 3 columnas con la información de la nueva categorización. Sin embargo, para la columa 'Surname' opté por eliminarla de nuestro dataset ya que al implementar la codificación OHE nos iba a crear un data set con una gran cantidad de columnas y el costo computacional seria muy alto una vez que comencemos a entrenar a nuestros modelos. Además, ahora no nos arroja nada de información, salvo el identificar al usuario pero en ese caso también podemos usar la columna 'Customer ID'

## Equilibrio de clases y primer entrenamiento del modelo

### Estandarización de clases

In [6]:
# Vamos a crear nuestros conjuntos de entrenamiento y de validación para el entrenamiento de nuestro modelo
# Vamos a estandarizas los valores de nuestras caracteristicas numéricas para evitar un sesgo del modelo

target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

numeric = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

print(features_train.shape)
print(features_train.head())
print(features_valid.shape)
print(features_valid.head())


(7500, 13)
      RowNumber  CustomerId  CreditScore       Age    Tenure   Balance  \
226         227    15774393     0.442805 -0.841274  1.445348 -1.224577   
7756       7757    15606232    -0.310897 -0.270730  0.718347  0.641783   
2065       2066    15581840    -0.259274 -0.556002  1.081847 -1.224577   
2800       2801    15646817     1.217157  1.155631  1.445348  1.290462   
7028       7029    15618410     0.690598 -1.221637 -0.000568  1.142121   

      NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
226        0.817772          1               1        -1.269750   
7756      -0.896874          1               1         0.960396   
2065       0.817772          1               0         0.661864   
2800       0.817772          1               0        -1.039476   
7028      -0.896874          0               0        -0.851729   

      Geography_Germany  Geography_Spain  Gender_Male  
226               False            False        False  
7756              False      

### Balanceo de clases

#### Ajuste de pesos de clase

In [7]:
# Vamos a entrenar al modelo con Regresión Logistica y evaluarlo para validar en que nivel de precisión y recall andamos

model = LogisticRegression(random_state=12345, solver='liblinear',)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print(confusion_matrix(target_valid, predicted_valid))
print("Recall del modelo =", recall_score(target_valid, predicted_valid))
print("Precision del modelo =", precision_score(target_valid, predicted_valid))
print("Valor F1 del modelo =", f1_score(target_valid, predicted_valid))

[[1965    0]
 [ 535    0]]
Recall del modelo = 0.0
Precision del modelo = 0.0
Valor F1 del modelo = 0.0


c:\Users\jrcis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
#Vamos a entrenar el mismo modelo pero usando el parametro class_weight para balancear las clases de la variable objetivo

model = LogisticRegression(random_state=12345, solver='liblinear',class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print(confusion_matrix(target_valid, predicted_valid))
print("Recall del modelo =", recall_score(target_valid, predicted_valid))
print("Precision del modelo =", precision_score(target_valid, predicted_valid))
print("Valor F1 del modelo =", f1_score(target_valid, predicted_valid))

[[1392  573]
 [ 159  376]]
Recall del modelo = 0.702803738317757
Precision del modelo = 0.39620653319283455
Valor F1 del modelo = 0.5067385444743935


#### Sobremuestreo

In [9]:
# Vamos a utilizar el método de Sobremuestreo para balancear las clases objetivo
from sklearn.utils import shuffle
def upsample(features, target, repeat):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]
    
    features_upsampled = pd.concat([features_zeros]+[features_ones]*repeat)
    target_upsampled = pd.concat([target_zeros]+[target_ones]*repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

print(features_upsampled.shape)
print(target_upsampled.shape)
print(target_upsampled.value_counts())

(12006, 13)
(12006,)
Exited
1    6008
0    5998
Name: count, dtype: int64


In [10]:
#Vamos a entrenar el modelo con Sobremuestreo y calcular sus metricas

model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print(confusion_matrix(target_valid, predicted_valid))
print("Recall del modelo =", recall_score(target_valid, predicted_valid))
print("Precision del modelo =", precision_score(target_valid, predicted_valid))
print("Valor F1 del modelo =", f1_score(target_valid, predicted_valid))

[[1001  964]
 [ 272  263]]
Recall del modelo = 0.491588785046729
Precision del modelo = 0.2143439282803586
Valor F1 del modelo = 0.2985244040862656


#### Umbral de clasificación

In [11]:
# Vamos a obtener las probabilidades para el umbral de clasificación para un modelo de Regresión Logística y vamos a obtener sus métricas
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:,1]

for threshold in np.arange(0,0.3,0.02):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    print('Threshold = {:.2f} | Precision = {:.3f}, Recall = {:.3f}'.format(threshold, precision, recall))
    
    

Threshold = 0.00 | Precision = 0.214, Recall = 1.000
Threshold = 0.02 | Precision = 0.214, Recall = 1.000
Threshold = 0.04 | Precision = 0.214, Recall = 1.000
Threshold = 0.06 | Precision = 0.214, Recall = 1.000
Threshold = 0.08 | Precision = 0.214, Recall = 1.000
Threshold = 0.10 | Precision = 0.214, Recall = 1.000
Threshold = 0.12 | Precision = 0.214, Recall = 1.000
Threshold = 0.14 | Precision = 0.214, Recall = 1.000
Threshold = 0.16 | Precision = 0.214, Recall = 1.000
Threshold = 0.18 | Precision = 0.214, Recall = 1.000
Threshold = 0.20 | Precision = 0.224, Recall = 0.607
Threshold = 0.22 | Precision = 0.000, Recall = 0.000
Threshold = 0.24 | Precision = 0.000, Recall = 0.000
Threshold = 0.26 | Precision = 0.000, Recall = 0.000
Threshold = 0.28 | Precision = 0.000, Recall = 0.000


c:\Users\jrcis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jrcis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jrcis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

#### Segunda conclusión

Para esta segunda parte, hemos estandarizado los datos dado que nuestras variables numéricas tienen valores en distintas escalas y queremos evitar que nuestro modelo se llegue a sesgar o le de mayor importancia a una variable que a otra por sus valores. Es por eso que usamos la función Standard Scaler antesd e cualqueir entrenamiento

Posteriormente, hemos notado que nuestra vairable objetivo 'Exited' esta desbalanceada. Para el conjunto de entrenamiento tenemos un 4:1 entre valores con '0' (cliente que no se han ido) y '1' (clientes que han dejado el banco). Necesitamos ejecutar técnicas de balanceo de clases para que nuestro modelo no sesgue sus predicciones después de su entrenamiento. ¿Como sabemos esto? Hemos corrido las predicciones, usando un modelo de Regresión Logistica, sin balancear las clases y los resultados de precisión y recall han sido de 0.0. ¡Inaceptable!

-> Ajuste de peso de clases: Primero usamos el parametro 'class_weigth' en el modelo de Regresión Logistica, que realiza un balanceo de la clase objetivo durante el entrenamiento.

-> Sobremuestreo: Segunda técnica, hemos creado nuevas observaciones que contengan '1' para nuestra variables objetivo para nivelar la cantidad de datos de las dos clases que tenemos.

-> Tercera técnica, hemos calculado las probabilidades de las clases. Sin un ajuste de este umbral, la probabilidad será de 0.5 para cada clase, en este caso como buscamos tener más valores positivos dado que está desbalanceada la clase, se juega con la probabilidad en un rango de 0 a 0.3.

Resultados: El ajuste de clases desde el parametro 'class_weigth' nos ha dado un mejor resultado al obtener un valor F1 de 0.5. Por su parte, en el ajuste de umbral, el mejor resutlado lo obtuvimos al setear una probabilidad de 0.3 para la clase negativa ya que onbtuvimos una precisión de 0.227 y un recall de 0.6. Por útlimo, el sobremuestreo nos arrojó un valor F1 DE 0.3, por debajo de el ajuste de peso de clases y del ajuste del umbral de probabilidades.

Nos quedamos con el ajuste de pesos de clases 'class_weigth' para seguir ahora mejorando el modelo y obtener un valor F1 mayor a 0.6.



## Méjora del modelo

### Árbol de decisión

In [12]:
# Vamos a entrenar a nuestro modelo usando Árboles de decisión

from sklearn.tree import DecisionTreeClassifier

f1 = 0
best_recall_score = 0
best_precision_score = 0

for trees in range (1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=trees, class_weight = 'balanced')
    model.fit(features_train, target_train)
    answer = model.predict(features_valid)
    f1_score_ = f1_score(target_valid, answer)
    recall_score_ = recall_score(target_valid, answer)
    precision_score_ = precision_score(target_valid, answer)
    if f1_score_ > f1:
        f1 = f1_score_
        max_depth = trees
        best_recall_score = recall_score_
        best_precision_Score = precision_score

print("Profundidad de arboles =", max_depth)
print("Recall del modelo =", best_recall_score)
print("Precision del modelo =", best_precision_score)
print("Valor F1 del modelo =", f1)

Profundidad de arboles = 5
Recall del modelo = 0.6934579439252336
Precision del modelo = 0
Valor F1 del modelo = 0.6008097165991902


### Bosque Aleatorio

In [13]:
#Vamos a entrenar a nuestro modelo usando un Bosque Aleatorio y a probar con distintos ajustes de hiperparametros
from sklearn.ensemble import RandomForestClassifier

f1 = 0
i= 0
best_recall_score = 0
best_precision_score = 0

for i in range(1,16):
    model = RandomForestClassifier(random_state=12345, n_estimators = 10, max_depth = i, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1_score_tree = f1_score(target_valid, predicted_valid)
    recall_score_tree = recall_score(target_valid, predicted_valid)
    precision_score_tree = precision_score(target_valid, predicted_valid)
    if f1_score_tree > f1:
        f1 = f1_score_tree
        max_depth = i
        best_recall_score = recall_score_tree
        best_precision_score = precision_score_tree

print("Profundidad de arboles =", max_depth)
print("Recall del modelo =", best_recall_score)
print("Precision del modelo =", best_precision_score)
print("Valor F1 del modelo =", f1)

Profundidad de arboles = 10
Recall del modelo = 0.6299065420560748
Precision del modelo = 0.6127272727272727
Valor F1 del modelo = 0.6211981566820276


In [14]:
# Vamos a probar moviendonos sobre disinto ajuste para N_estimators

f1 = 0
best_recall_score = 0
best_precision_score = 0

for i in range(5,50, 5):
    model = RandomForestClassifier(random_state=12345, n_estimators = i, max_depth = 10, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1_score_tree = f1_score(target_valid, predicted_valid)
    recall_score_tree = recall_score(target_valid, predicted_valid)
    precision_score_tree = precision_score(target_valid, predicted_valid)
    if f1_score_tree > f1:
        f1 = f1_score_tree
        max_depth = i
        best_recall_score = recall_score_tree
        best_precision_score = precision_score_tree

print("Valor para hiperparametro n_estimators =", max_depth)
print("Recall del modelo =", best_recall_score)
print("Precision del modelo =", best_precision_score)
print("Valor F1 del modelo =", f1)

Valor para hiperparametro n_estimators = 30
Recall del modelo = 0.6299065420560748
Precision del modelo = 0.6406844106463878
Valor F1 del modelo = 0.6352497643732328


### Tercera conclusión

Hemos probado disintos algoritmos de aprendizaje para nuestro modelo y hemos jugado con sus parametros para encontrar la mejor combinación de estos que nos ayudará a encontrar el mayor valor de F1, que es la media armonizada de la precisión y el recall.

--> El módelo de Logistic Regression si bien es de un costo computacional bajo y se ejecuta de una manera muy rápida, al ejecutarlo sin un balanceo de clases nos arrroja un recall y precisión de 0,0. Al establecer el parametro weigth_class, mejora muchisimo al modelo. Sin embargo el valor F1 es de 0.5, se debe seguir mejorando.

--> El modelo de Arbol de decisión, nos arroja un recall de 0.69, muy bueno, sin embargo la precisión s de 0.0. Una clara evidencia de que nuestro modelo esta SOBREAJUSTADO ya que nos dice que nuestro modelo es capaz de identificar muy bien los Valores Positivos entre todos los valores posibles pero no es capaz de hacer una buena predicción.

--> Bosque Aleatorio ha sido nuestro MEJOR MODELO, ya que obtuvimos un valor F1 de 0.62 con valores de Recall y Precision muy similares. Estas metricas se alcanzaron usando hiperparametros como max_depth y n_estimators. Se realziaron bucles para encontrar los mejores valores para nuestros hiperparametros, la cantidad de estimadores se fijo en ser menor que 50 para evitar un costo computacional muy alto y la profundiad del modelo de estableció menor a 10 por el sobreajuste de nuestro modelo.

## Prueba final

### Ejecución del mejor modelo

In [15]:
# Finalmente vamos a entrenar nuestro modelo con el algoritmo y parametros con los que obtuvimos mejores resultados

model = RandomForestClassifier(random_state=12345, n_estimators = 40, max_depth=10, class_weight = 'balanced')
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
f1_score_final = f1_score(target_valid, predicted_valid)
recall_score_final = recall_score(target_valid, predicted_valid)
precision_score_final = precision_score(target_valid, predicted_valid)

print(f"El mejor modelo entrenado para determinar si los clientes de Beta Bank se irán o no tiene un recall de {recall_score_final}, una precisión de {precision_score_final} y un valor F1 de {f1_score_final}.")

# Por ultimo obtenemos el valor AUC-ROC (Area bajo la Curva) para valdiar los resultados de nuestro modelo

probabilities = model.predict_proba(features_valid)
probabilities_one = probabilities[:,1]

auc_roc = roc_auc_score(target_valid, probabilities_one)
print(f"El valor del área bajo la curva ROC, que va de 0 a 1, es de: {auc_roc}.")


El mejor modelo entrenado para determinar si los clientes de Beta Bank se irán o no tiene un recall de 0.6299065420560748, una precisión de 0.6346516007532956 y un valor F1 de 0.6322701688555347.
El valor del área bajo la curva ROC, que va de 0 a 1, es de: 0.8636037192932391.


### Conclusión Final

¡Los resultados de nuestro modelo han sido satisfactorios!

Para esto hemos utilizado un algoritmo de aprendizaje de Bosque Aleatorio para nuestro modelo que presentaba un sobreajuste y un desbalanceo de clases en un principio. Temas que se abordaron con el uso de hiperparametros como class_weight, n_estimators y max_depth.

El valor de F1 es de 0.62 para nuestro conjunto de validación, mientras que el valor de AUC-ROC es de 0.86, valor que está por encima del 0.5, lo que coincide con los valores de precisión y recall que tenemos en nuestro modelo (Verdaderos Positivos vs Flalsos Positivos)

Ahora podemos ejecutar nuestro modelo para poder encontrar a aquellos clientes que, con un 62% de precisiómn, podemos decir que se irán del banco. Y para ellos, nuestro equipo de marketing podra trabajar en una estrtaegia especifica para abordar aquellas necesidades o carencias que tienen en su servicio y podamos revertir una potencial decisión de huida de capital del banco Beta Bank